<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Upload files for HTML User Interface

In the notebook [1_image_similarity_export.ipynb](1_image_similarity_export.ipynb) we exported reference image features, reference image file names, and reference image thumbnails. In this notebook we upload those items, as well as our simplified HTML interface, to your Azure Blob storage account for easy public access. 

You should create an Azure Storage Account and a "Container" in that account to store your uploaded files.  

## Initialization

In [1]:
# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# Standard python libraries
import sys
from pathlib import Path
from tqdm.notebook import trange, tqdm
import os, uuid
import azure.storage.blob
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, ContentSettings

# Check Storage SDK version number
print(f"Azure Blob Storage SDK Version: {azure.storage.blob.VERSION}")

Azure Blob Storage SDK Version: 12.2.0


First we setup variables to point to your Azure Blob storage account and your existing Blob container. May be best to setup a fresh Blob container for this upload. 

In [3]:
AZURE_ACCOUNT_NAME = "YOUR ACCOUNT NAME"
AZURE_ACCOUNT_KEY = "YOUR ACCOUNT ACCESS KEY"
BLOB_CONTAINER_NAME = "YOUR CONTAINER NAME"
ENDPOINT_SUFFIX = "core.windows.net"

Next we upload the files to your Azure Blob storage.

In [4]:
azure_storage_connection_str = "DefaultEndpointsProtocol=https;AccountName={};AccountKey={};EndpointSuffix={}".format(AZURE_ACCOUNT_NAME, AZURE_ACCOUNT_KEY, ENDPOINT_SUFFIX)
container_name = BLOB_CONTAINER_NAME
local_files = ['../UICode/data/ref_filenames.zip','../UICode/data/ref_features.zip','../UICode/index.html','../UICode/script.js','../UICode/example_imgs.js','../UICode/style.css']
blob_files = ['data/ref_filenames.zip','data/ref_features.zip','index.html','script.js','example_imgs.js','style.css']

# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(azure_storage_connection_str)

# Get total size of non-image files to upload
sizecounter = 0
for file in local_files:
    sizecounter += os.stat(file).st_size
    
print("Uploading non-image files:")

# # Upload the individual files for the front-end and the ZIP files for reference features
i = 0
with tqdm(total=sizecounter, unit='B', unit_scale=True, unit_divisor=1024) as pbar:
    while (i < len(local_files)):
        # Create a blob client using the local file name as the name for the blob
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_files[i])

        # Upload the file
        with open(local_files[i], "rb") as data:
            buf = 0
            buf = os.stat(local_files[i]).st_size
            if (i==2):
                blob_client.upload_blob(data, overwrite=True, content_settings=ContentSettings(content_type="text/html"))
            elif (i==5):
                blob_client.upload_blob(data, overwrite=True, content_settings=ContentSettings(content_type="text/css"))
            else:
                blob_client.upload_blob(data, overwrite=True)
            if buf:
                pbar.update(buf)
        i+=1

# Upload the thumbnail versions of the reference images
path_blob = 'small-150'
path_local = '../UICode/{}'.format(path_blob)

# Get total size of all image files to upload
sizecounter = 0
for root, dirs, files in os.walk(path_local):
    for file in files:
        sizecounter += os.stat(os.path.join(path_local, file)).st_size

print("Uploading thumbnail image files:")

with tqdm(total=sizecounter, unit='B', unit_scale=True, unit_divisor=1024) as pbar:
    for root, dirs, files in os.walk(path_local):
        for file in files:
            blob_client = blob_service_client.get_blob_client(container=container_name, blob=path_blob+'/'+file)
            with open(os.path.join(path_local, file), "rb") as data:
                buf = 0
                buf = os.stat(os.path.join(path_local, file)).st_size
                blob_client.upload_blob(data, overwrite=True)
                if buf:
                    pbar.set_postfix(file=file, refresh=False)
                    pbar.update(buf)


Uploading non-image files:



Uploading thumbnail image files:
